## Step 1: Load Processed Review File

In [2]:
# download the dataset
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Electronics.json.gz

--2024-12-18 21:59:53--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Electronics.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3322874357 (3.1G) [application/x-gzip]
Saving to: ‘Electronics.json.gz’

Electronics.json.gz 100%[===================>]   3.09G  47.5MB/s    in 92s     

2024-12-18 22:01:25 (34.4 MB/s) - ‘Electronics.json.gz’ saved [3322874357/3322874357]



In [9]:
import gzip
import json
import re
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import random

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Define stopwords
stop_words = set(stopwords.words('english'))

# Text preprocessing function
def preprocess_text(review):
    text = review.lower()  # Lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenize
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

# Step 1: Load reviews and ratings
file_path = "Electronics.json.gz"
reviews = []
ratings = []

with gzip.open(file_path, 'rt') as f:
    for i, line in enumerate(f):
        review = json.loads(line)
        if 'reviewText' in review and 'overall' in review:
            reviews.append(review['reviewText'])
            ratings.append(review['overall'])
        if i >= 100000:  # Limit to first 100,000 reviews
            break

print(f"Loaded {len(reviews)} reviews and {len(ratings)} ratings.")

# Step 2: Randomly select 10% of the data
subset_size = int(0.1 * len(reviews))  # Calculate 10% of the total data
subset_indices = random.sample(range(len(reviews)), subset_size)

# Create the 10% subset
reviews_subset = [reviews[i] for i in subset_indices]
ratings_subset = [ratings[i] for i in subset_indices]

print(f"Subset size: {len(reviews_subset)} reviews and {len(ratings_subset)} ratings.")

# Step 3: Preprocess the reviews
print("Starting parallel preprocessing...")
with ProcessPoolExecutor() as executor:
    processed_reviews_subset = list(tqdm(executor.map(preprocess_text, reviews_subset), total=len(reviews_subset)))

print("Preprocessing complete.")

# Step 4: Combine into a DataFrame
data = pd.DataFrame({
    'original_review': reviews_subset,
    'processed_review': processed_reviews_subset,
    'overall': ratings_subset
})

# Step 5: Save the final DataFrame
data.to_csv('processed_reviews_10percent.csv', index=False)
print("File saved: processed_reviews_10percent.csv")

# Verify
print(data.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Loaded 99987 reviews and 99987 ratings.
Subset size: 9998 reviews and 9998 ratings.
Starting parallel preprocessing...


100%|██████████| 9998/9998 [00:04<00:00, 2141.29it/s]


Preprocessing complete.
File saved: processed_reviews_10percent.csv
                                     original_review  \
0  Yes, you get what you pay for, but I've had ch...   
1  Best $5 I have ever spent.  I should have boug...   
2                                         not usable   
3            Always useful, this one for camera use.   
4  Great product, able to hide it when i don't ne...   

                                    processed_review  overall  
0  yes get pay ive cheap headphones last many yea...      2.0  
1  best 5 ever spent bought 2 great listening lat...      5.0  
2                                             usable      1.0  
3                       always useful one camera use      5.0  
4  great product able hide dont need break im goi...      5.0  


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

# Step 1: Load the Data
data = pd.read_csv('processed_reviews_10percent.csv')

# Step 2: Create Sentiment Labels
def assign_sentiment(rating):
    if rating >= 4:
        return 0  # Positive
    elif rating <= 2:
        return 1  # Negative
    else:
        return 2  # Neutral

data['sentiment'] = data['overall'].apply(assign_sentiment)

# Drop unnecessary columns
data = data[['processed_review', 'sentiment']]

In [12]:
# Step 3: Split into Train, Validation, and Test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

print(f"Train size: {len(train_data)}, Validation size: {len(val_data)}, Test size: {len(test_data)}")


Train size: 7198, Validation size: 800, Test size: 2000


In [13]:
# Step 4: Vocabulary (from Part 1)
UNK_IDX = 0
PAD_IDX = 1

def text_to_indices(text, vocab, max_len=50):
    tokens = text.split()  # Preprocessed reviews are space-separated
    indices = [vocab.get(token, UNK_IDX) for token in tokens]
    return indices[:max_len] + [PAD_IDX] * (max_len - len(indices))

In [14]:
from collections import Counter

# Step 1: Clean the 'processed_review' column
train_data = train_data.dropna(subset=['processed_review']).reset_index(drop=True)
train_data['processed_review'] = train_data['processed_review'].astype(str)

# Step 2: Build the vocabulary
min_word_count = 5  # Words must appear at least 5 times to be included
counter = Counter()

# Count word frequencies in the 'processed_review' column
for review in train_data['processed_review']:
    counter.update(review.split())

# Create the vocabulary
vocab = {word: idx + 2 for idx, (word, count) in enumerate(counter.items()) if count >= min_word_count}
vocab['<PAD>'] = 0  # Padding index
vocab['<UNK>'] = 1  # Unknown words index

print(f"Vocabulary Size: {len(vocab)}")


Vocabulary Size: 4758


In [15]:
print("Sample vocabulary:", list(vocab.items())[:10])


Sample vocabulary: [('bought', 2), ('noise', 3), ('cancelling', 4), ('headset', 5), ('use', 6), ('airplane', 7), ('make', 8), ('easier', 9), ('control', 10), ('volume', 11)]


In [16]:
# Clean the 'processed_review' column in all datasets
for df in [train_data, val_data, test_data]:
    df.dropna(subset=['processed_review'], inplace=True)  # Drop rows with NaN
    df['processed_review'] = df['processed_review'].astype(str).str.strip()  # Convert to strings and strip whitespaces
    df = df.reset_index(drop=True)  # Reset index after dropping rows

In [17]:
print("Train data sample:")
print(train_data['processed_review'].head())

print("Validation data sample:")
print(val_data['processed_review'].head())

print("Test data sample:")
print(test_data['processed_review'].head())


Train data sample:
0    bought noise cancelling headset use airplane m...
1           needed review cassette tapes product ideal
2    mighty 1200 live forever yes sound better cd p...
3       unable justify rating used need longer concern
4    typical romance novel nothing fantastic easy r...
Name: processed_review, dtype: object
Validation data sample:
7614    good job holding 55 lcd like way swivels folds...
9314    nice set wires love purchase better ones buy 9...
8785    set easy works well one got spectrum problem f...
8746                                                sweet
9207    beauty card rj45 receptacle attached permanent...
Name: processed_review, dtype: object
Test data sample:
4122    theyre perfect far appear sturdy sound great t...
4065    perfect set tools need flashlight overall sati...
1731    bought child tested first normally earphones h...
4740                                    satisfied product
6391    purchased tv mount tv motorhome several friend...
Name: pr

In [18]:
def text_to_indices(text, vocab, max_len=50):
    if not isinstance(text, str):  # Ensure text is a string
        text = ""
    tokens = text.split()
    indices = [vocab.get(token, vocab['<UNK>']) for token in tokens]
    return indices[:max_len] + [vocab['<PAD>']] * (max_len - len(indices))


In [19]:
from torch.utils.data import DataLoader, Dataset
# Step 5: Dataset Class
# Ensure SentimentDataset class is defined correctly
class SentimentDataset(Dataset):
    def __init__(self, data, vocab, max_len=50):
        self.vocab = vocab
        self.reviews = [text_to_indices(text, vocab, max_len) for text in data['processed_review']]
        self.labels = data['sentiment'].values.astype(int)  # Ensure labels are integers

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        processed_review = [
            self.vocab.get(token, 1)  # Use 1 as UNK_IDX if token is not in vocab
            for token in self.reviews[idx]
        ]


        # Convert to tensors
        return torch.tensor(processed_review, dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

# Recreate datasets and DataLoaders
train_dataset = SentimentDataset(train_data, vocab, max_len=50)
val_dataset = SentimentDataset(val_data, vocab, max_len=50)
test_dataset = SentimentDataset(test_data, vocab, max_len=50)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=2)

In [21]:
import torch.nn as nn

class SentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx):
        super(SentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # Output logits for 3 classes

    def forward(self, x):
        embedded = self.embedding(x)  # Shape: [batch_size, seq_len, embedding_dim]
        embedded = embedded.mean(dim=1)  # Average over sequence length
        hidden = self.relu(self.fc1(embedded))
        output = self.fc2(hidden)  # Shape: [batch_size, num_classes]
        return output

In [22]:
for reviews, labels in train_loader:
    print("Reviews shape:", reviews.shape)
    print("Labels shape:", labels.shape)
    print("Min index:", reviews.min(), "Max index:", reviews.max())
    break

Reviews shape: torch.Size([32, 50])
Labels shape: torch.Size([32])
Min index: tensor(1) Max index: tensor(1)


In [23]:
print("Sample vocab entries:", list(vocab.items())[:10])  # Print the first 10 vocab entries
print("Vocabulary size:", len(vocab))


Sample vocab entries: [('bought', 2), ('noise', 3), ('cancelling', 4), ('headset', 5), ('use', 6), ('airplane', 7), ('make', 8), ('easier', 9), ('control', 10), ('volume', 11)]
Vocabulary size: 4758


In [24]:
print("Sample processed review:", train_data['processed_review'].iloc[0])
tokens = train_data['processed_review'].iloc[0].split()
print("Tokens in sample review:", tokens)

# Check if tokens are in vocab
for token in tokens:
    if token in vocab:
        print(f"Token '{token}' is in vocab with index {vocab[token]}")
    else:
        print(f"Token '{token}' is NOT in vocab")


Sample processed review: bought noise cancelling headset use airplane make easier control volume without dig ipod switches titlestracks works fine downside cordcable wish cable could retracted desired length
Tokens in sample review: ['bought', 'noise', 'cancelling', 'headset', 'use', 'airplane', 'make', 'easier', 'control', 'volume', 'without', 'dig', 'ipod', 'switches', 'titlestracks', 'works', 'fine', 'downside', 'cordcable', 'wish', 'cable', 'could', 'retracted', 'desired', 'length']
Token 'bought' is in vocab with index 2
Token 'noise' is in vocab with index 3
Token 'cancelling' is in vocab with index 4
Token 'headset' is in vocab with index 5
Token 'use' is in vocab with index 6
Token 'airplane' is in vocab with index 7
Token 'make' is in vocab with index 8
Token 'easier' is in vocab with index 9
Token 'control' is in vocab with index 10
Token 'volume' is in vocab with index 11
Token 'without' is in vocab with index 12
Token 'dig' is NOT in vocab
Token 'ipod' is in vocab with inde

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SentimentClassifier(vocab_size=len(vocab), embedding_dim=50, hidden_dim=100, output_dim=3, pad_idx=0)
model = model.to(device)  # Move the model to the selected device

for reviews, labels in train_loader:
    # Move input tensors to the model's device
    reviews, labels = reviews.to(device), labels.to(device)
    print("Input reviews device:", reviews.device)  # Debug check
    print("Labels device:", labels.device)
    print("Model device:", next(model.parameters()).device)

    # Forward pass
    outputs = model(reviews)
    print("Outputs device:", outputs.device)  # Confirm outputs are on the same device
    break


Input reviews device: cuda:0
Labels device: cuda:0
Model device: cuda:0
Outputs device: cuda:0


In [26]:
import torch
import torch.optim as optim

# Model parameters
vocab_size = len(vocab)  # Size of vocabulary
embedding_dim = 128      # Dimension of word embeddings
hidden_dim = 128         # Hidden dimension size
output_dim = 3           # Number of classes: Positive, Neutral, Negative
PAD_IDX = vocab['<PAD>'] # Padding index

# Initialize the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SentimentClassifier(vocab_size, embedding_dim, hidden_dim, output_dim, PAD_IDX).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [27]:
def train_model(model, train_loader, val_loader, optimizer, criterion, epochs=5):
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        train_loss = 0

        for reviews, labels in train_loader:
            reviews, labels = reviews.to(device), labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            predictions = model(reviews)

            # Compute loss
            loss = criterion(predictions, labels)
            train_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation step
        model.eval()
        val_loss = 0
        correct, total = 0, 0

        with torch.no_grad():
            for reviews, labels in val_loader:
                reviews, labels = reviews.to(device), labels.to(device)
                predictions = model(reviews)
                loss = criterion(predictions, labels)
                val_loss += loss.item()

                # Compute accuracy
                preds = torch.argmax(predictions, dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        # Print epoch results
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss / len(train_loader):.4f}")
        print(f"Validation Loss: {val_loss / len(val_loader):.4f}")
        print(f"Validation Accuracy: {correct / total:.4f}")


In [28]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=4, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [29]:
gradient_accumulation_steps = 2  # Accumulate gradients for 2 batches
optimizer.zero_grad()

for i, (reviews, labels) in enumerate(train_loader):
    reviews, labels = reviews.to(device), labels.to(device)
    outputs = model(reviews)
    loss = criterion(outputs, labels)
    loss.backward()

    if (i + 1) % gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()

In [30]:
print("Unique labels in train data:", train_data['sentiment'].unique())
print("Unique labels in validation data:", val_data['sentiment'].unique())
print("Unique labels in test data:", test_data['sentiment'].unique())

Unique labels in train data: [0 2 1]
Unique labels in validation data: [0 1 2]
Unique labels in test data: [0 2 1]


In [31]:
train_model(model, train_loader, val_loader, optimizer, criterion, epochs=5)

Epoch 1/5
Train Loss: 0.6622
Validation Loss: 0.6762
Validation Accuracy: 0.7779
Epoch 2/5
Train Loss: 0.6638
Validation Loss: 0.6746
Validation Accuracy: 0.7779
Epoch 3/5
Train Loss: 0.6630
Validation Loss: 0.6761
Validation Accuracy: 0.7779
Epoch 4/5
Train Loss: 0.6610
Validation Loss: 0.6764
Validation Accuracy: 0.7779
Epoch 5/5
Train Loss: 0.6619
Validation Loss: 0.6750
Validation Accuracy: 0.7779


In [32]:
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for reviews, labels in test_loader:
            reviews, labels = reviews.to(device), labels.to(device)
            predictions = model(reviews)
            preds = torch.argmax(predictions, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    test_accuracy = correct / total
    print(f"Test Accuracy: {test_accuracy:.4f}")

# Evaluate the model
evaluate_model(model, test_loader)

Test Accuracy: 0.7944


## Part 3: Extensions to the Model (6 Points)

In [33]:
# Trying different embedding dimensions
for embedding_dim in [50, 100, 200]:
    print(f"Training with embedding dimension: {embedding_dim}")

    model = SentimentClassifier(
        vocab_size=len(vocab),
        embedding_dim=embedding_dim,  # Variable embedding dimension
        hidden_dim=100,              # First hidden layer dimension
        output_dim=3,                 # Number of classes
        pad_idx=0                     # Padding index
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(5):
        model.train()
        total_loss = 0

        for reviews, labels in train_loader:
            reviews, labels = reviews.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(reviews)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for reviews, labels in val_loader:
            reviews, labels = reviews.to(device), labels.to(device)
            outputs = model(reviews)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy with embedding dim {embedding_dim}: {correct / total:.4f}")


Training with embedding dimension: 50
Epoch 1, Loss: 0.6727568133672078
Epoch 2, Loss: 0.6617682127157847
Epoch 3, Loss: 0.6619373570548164
Epoch 4, Loss: 0.6606744694709777
Epoch 5, Loss: 0.6600558445188734
Validation Accuracy with embedding dim 50: 0.7779
Training with embedding dimension: 100
Epoch 1, Loss: 0.6715475738048553
Epoch 2, Loss: 0.6634536892175674
Epoch 3, Loss: 0.6610300470723046
Epoch 4, Loss: 0.6612279424402449
Epoch 5, Loss: 0.6602384403016832
Validation Accuracy with embedding dim 100: 0.7779
Training with embedding dimension: 200
Epoch 1, Loss: 0.6705974475542704
Epoch 2, Loss: 0.664615275727378
Epoch 3, Loss: 0.6634033865398831
Epoch 4, Loss: 0.6627227024237314
Epoch 5, Loss: 0.6626748592323727
Validation Accuracy with embedding dim 200: 0.7779


In [34]:
#Adding a Hidden Layer
class SentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, output_dim, pad_idx):
        super(SentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc1 = nn.Linear(embedding_dim, hidden_dim1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)  # New hidden layer
        self.fc3 = nn.Linear(hidden_dim2, output_dim)   # Output layer

    def forward(self, x):
        embedded = self.embedding(x).mean(dim=1)  # Average embeddings
        hidden1 = self.relu(self.fc1(embedded))
        hidden2 = self.relu(self.fc2(hidden1))  # Pass through second hidden layer
        output = self.fc3(hidden2)
        return output

embedding_dim=100
model = SentimentClassifier(
vocab_size=len(vocab),
embedding_dim=embedding_dim,
hidden_dim1=128,  # First hidden layer
hidden_dim2=64,   # Second hidden layer
output_dim=3,
pad_idx=0
).to(device)

# Train the model
for epoch in range(5):  # Train for 5 epochs
    model.train()
    total_loss = 0
    for reviews, labels in train_loader:
        reviews, labels = reviews.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(reviews)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

# Evaluate on validation set
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for reviews, labels in val_loader:
        reviews, labels = reviews.to(device), labels.to(device)
        outputs = model(reviews)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy with embedding dim {embedding_dim}: {correct / total:.4f}")


Epoch 1, Loss: 0.9999055385589599
Epoch 2, Loss: 0.9999153282907274
Epoch 3, Loss: 0.9998822771178352
Epoch 4, Loss: 0.9998394544919332
Epoch 5, Loss: 0.9999085876676771
Validation Accuracy with embedding dim 100: 0.7779


In [35]:
#Freeze Embeddings
class SentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, output_dim, pad_idx):
        super(SentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.embedding.weight.requires_grad = False  # Freeze embeddings
        self.fc1 = nn.Linear(embedding_dim, hidden_dim1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        embedded = self.embedding(x).mean(dim=1)  # Average embeddings
        hidden1 = self.relu(self.fc1(embedded))
        hidden2 = self.relu(self.fc2(hidden1))
        output = self.fc3(hidden2)
        return output

#Train the model
model = SentimentClassifier(
    vocab_size=len(vocab),
    embedding_dim=100,
    hidden_dim1=256,
    hidden_dim2=128,
    output_dim=3,
    pad_idx=0
).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# Train the model
for epoch in range(5):  # Train for 5 epochs
    model.train()
    total_loss = 0
    for reviews, labels in train_loader:
        reviews, labels = reviews.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(reviews)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

#Evaluate the model
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for reviews, labels in val_loader:
        reviews, labels = reviews.to(device), labels.to(device)
        outputs = model(reviews)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy with frozen embeddings: {correct / total:.4f}")

Epoch 1, Loss: 0.6695410244994693
Epoch 2, Loss: 0.6657237780094146
Epoch 3, Loss: 0.663375201092826
Epoch 4, Loss: 0.6616350891855028
Epoch 5, Loss: 0.6614855986171299
Validation Accuracy with frozen embeddings: 0.7779


The validation accuracy remains the same (07779) for both frozen and fine-tuned embeddings.

The pre-trained embeddings may already represent the input text well enough for the task. Further tuning doesn't add significant value.

Freezing the pre-trained embeddings is a more computationally efficient choice since it yields the same performance as fine-tuning (0.7779 validation accuracy) while requiring fewer parameters to update.

## Part 4: Comparison with Random Embeddings

In [36]:
import torch.nn as nn

# Define a new model with random embeddings
class SentimentClassifierRandom(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx):
        super(SentimentClassifierRandom, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        # Initialize with random weights
        nn.init.uniform_(self.embedding.weight, -0.1, 0.1)
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x).mean(dim=1)  # Average embeddings
        hidden = self.relu(self.fc1(embedded))
        output = self.fc2(hidden)
        return output

# Initialize the model
model_random = SentimentClassifierRandom(
    vocab_size=len(vocab),
    embedding_dim=100,
    hidden_dim=128,
    output_dim=3,
    pad_idx=0
).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_random.parameters(), lr=0.001)

# Training loop
for epoch in range(5):  # Train for 5 epochs
    model_random.train()
    total_loss = 0

    for reviews, labels in train_loader:
        reviews, labels = reviews.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_random(reviews)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

model_random.eval()
correct, total = 0, 0

with torch.no_grad():
    for reviews, labels in test_loader:
        reviews, labels = reviews.to(device), labels.to(device)
        outputs = model_random(reviews)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy with Random Embeddings: {correct / total:.4f}")


Epoch 1, Loss: 0.7078133289019267
Epoch 2, Loss: 0.6603058771292368
Epoch 3, Loss: 0.6597366638978323
Epoch 4, Loss: 0.6598447374502818
Epoch 5, Loss: 0.6597351370917426
Test Accuracy with Random Embeddings: 0.7944


Based on the output, the test accuracy for both models—one using pre-trained embeddings and the other using randomly initialized embeddings—was 0.7944. The sentiment classification task might not require complex word representations, so the model can achieve good accuracy with random embeddings.

Either that or, the training was unsuccessful.